In [1]:
%load_ext autoreload
%autoreload 2
%pylab inline

%pylab is deprecated, use %matplotlib inline and import the required libraries.
Populating the interactive namespace from numpy and matplotlib


In [2]:
import sys
sys.path.append("..")

In [3]:
from encoding.parse_rooms import parse_rooms, parse_exits, dictify_rooms

In [4]:
rooms = parse_rooms("../encoding/dsl/rooms_for_alloy.txt")
exits = parse_exits("../encoding/dsl/exits_for_alloy.txt")
design = dictify_rooms(rooms, exits)

In [5]:
from rom_tools.rom_manager import RomManager
from abstraction_validation.sm_paths import *

In [6]:
from bdds.bdd_core import *

In [7]:
from functools import reduce

In [8]:
all_nodes = []
for r, room in rooms.items():
    for node in room.graph.name_node.keys():
        all_nodes.append(node)
node_ids = {n:i for i,n in enumerate(all_nodes)}

In [9]:
# Issue with design translation via chopping off the last _
node_ids["Spore_Spawn_Spawn"] = node_ids["Spore_Spawn_Spore_Spawn"]
node_ids["Golden_Torizo_Torizo"] = node_ids["Golden_Torizo_Golden_Torizo"]
node_ids["Mother_Brain_Brain"] = node_ids["Mother_Brain_Mother_Brain"]

In [10]:
def combine_bdd_group(bdds):
    return reduce(lambda x,y: x|y, bdds, context_id.false)

In [11]:
def loc_id(room_name, node_name, when="prev"):
    node_id = node_ids[f"{room_name}_{node_name}"]
    return f"node_id_{when} = {node_id}"

In [12]:
def item_transitions(item_gained=None):
    if item_gained is None:
        return "(items_unchanged)"
    clauses = []
    for i in design["Items"] | design["Bosses"]:
        if i == item_gained:
            clause = f"{i}_prev < {i}_next"
        else:
            clause = f"{i}_prev = {i}_next"
        clauses.append(clause)
    return " & ".join(clauses)


def itemset_to_str(itemset):
    if len(itemset) == 0:
        return "TRUE"
    else:
        return " & ".join([f"{item}_prev = 1" for item in itemset])

def required_itemsets(itemsets):
    return "(" + " | ".join([itemset_to_str(itemset) for itemset in itemsets]) + ")"

In [13]:
# Only encode node_id and items
def mk_context_id():
    context = Context()
    context.declare(
        node_id_prev = (0,len(node_ids)),
        node_id_next = (0,len(node_ids)),
        node_id_goal = (0,len(node_ids)),
        node_id_temp = (0,len(node_ids)),
    )
    # Try to get a favorable variable ordering
    shared_dict = {}
    for i in item_mapping:
        shared_dict[f"{i}_prev"] = (0,1)
        shared_dict[f"{i}_next"] = (0,1)
        shared_dict[f"{i}_goal"] = (0,1)
        shared_dict[f"{i}_temp"] = (0,1)
    context.declare(**shared_dict)
    context.define(mk_items_unchanged())
    return context

In [14]:
context_id = mk_context_id()

In [15]:
def mk_goal_satisfied_id(context):
    # Expression that holds when the goal is satisfied (for all possible goals)
    goal_satisfied = context.true
    for v1, v2 in zip(nexts, goals):
        sat = context.add_expr(f"{v1} = {v2}")
        #print(sat.dag_size)
        #print(goal_satisfied.dag_size)
        goal_satisfied &= sat
    return goal_satisfied

In [16]:
# Have to refine some stuff from bdd_core for the new context
prev_to_temp = mk_replace(context_id, "prev", "temp")
next_to_temp = mk_replace(context_id, "next", "temp")
goal_to_next = mk_replace(context_id, "goal", "next")
prev_to_next = mk_replace(context_id, "prev", "next")
next_to_prev = mk_replace(context_id, "next", "prev")
temps = [k for k in context_id.vars.keys() if k.endswith("_temp")]
nexts = [k for k in context_id.vars.keys() if k.endswith("_next")]
prevs = [k for k in context_id.vars.keys() if k.endswith("_prev")]
goals = [k for k in context_id.vars.keys() if k.endswith("_goal")]

def mk_goal_satisfied_id(context):
    # Expression that holds when the goal is satisfied (for all possible goals)
    goal_satisfied = context.true
    for v1, v2 in zip(nexts, goals):
        sat = context.add_expr(f"{v1} = {v2}")
        #print(sat.dag_size)
        #print(goal_satisfied.dag_size)
        goal_satisfied &= sat
    return goal_satisfied

def mk_policy_id(trans_norule, context, goal_bdd=None):
    n = 0
    # prev, next, goal
    policy = context.false
    covered = mk_goal_satisfied_id(context)
    if goal_bdd:
        covered &= goal_bdd
    covered_last = context.false
    while covered != covered_last:
        # States with edges into a covered state
        covered_last = covered
        covered_temp = context.exist(nexts, trans_norule & covered)
        covered_temp = context.let(prev_to_next, covered_temp)
        # policy(prev, next, goal) = trans(prev, next) & covered(next, goal) & ~covered(prev, goal)
        covered_prev_goal = context.let(next_to_prev, covered)
        policy |= trans_norule & covered & ~covered_prev_goal
        # Find a state in covered that the fringe state transitions to
        covered |= covered_temp
        print(n, covered_temp.dag_size, covered.dag_size, policy.dag_size)
        n += 1
    return policy

def mk_policy_id2(trans_pn, context, goal_bdd=None, maxn=None):
    n = 0
    if maxn is None:
        maxn = float("inf")
    # prev, next, goal
    policy_png = context.false
    covered_ng = mk_goal_satisfied_id(context)
    if goal_bdd:
        covered_ng &= goal_bdd
    print(covered_ng.count())
    covered_last_ng = context.false
    while covered_ng != covered_last_ng and n < maxn:
        # States with edges into a covered state
        covered_last_ng = covered_ng
        fringe_pg = context.exist(nexts, trans_pn & covered_ng)
        fringe_ng = context.let(prev_to_next, fringe_pg) & ~covered_ng
        print(fringe_ng.count())
        print(context.exist(nonidns, fringe_ng).count())
        print(list(context.pick_iter(context.exist(nonidns, fringe_ng))))
        # policy(prev, next, goal) = trans(prev, next) & fringe(next, goal) & ~covered(prev, goal)
        policy_png |= trans_pn & fringe_ng
        # Find a state in covered that the fringe state transitions to
        covered_ng |= fringe_ng
        print(n, fringe_ng.dag_size, covered_ng.dag_size, policy_png.dag_size)
        n += 1
    return policy_png

In [17]:
def mk_policy_id3(trans_pn, context, goal_bdd=None, maxn=None):
    n = 0
    if maxn is None:
        maxn = float("inf")
    # prev, next, goal
    policy_png = context.false
    covered_ng = mk_goal_satisfied_id(context)
    if goal_bdd:
        covered_ng &= goal_bdd
    covered_last_ng = context.false
    while covered_ng != covered_last_ng and n < maxn:
        # States with edges into a covered state
        covered_last_ng = covered_ng
        covered_pg = context.let(next_to_prev, covered_ng)
        fringe_png = trans_pn & covered_ng & ~covered_pg
        # TODO: consider gluing the fringes together later?
        policy_png |= fringe_png
        #print(fringe_png.count())
        #print(context.exist(nonidns, fringe_png).count())
        #print(list(context.pick_iter(context.exist(nonidns, fringe_png))))
        # Update covered to include the prevs of fringe
        covered_ng |= context.let(prev_to_next, context.exist(nexts, fringe_png))
        print(n, fringe_png.dag_size, covered_ng.dag_size, policy_png.dag_size)
        n += 1
    return policy_png

In [18]:
# Build individual BDDs
room_bdds = []
door_bdds = []
for room_name, room in tqdm(design['Rooms'].items()):
    links = []
    for node_name in room['Nodes']:
        s = loc_id(room_name, node_name) + " & " + loc_id(room_name, node_name, when="next") + " & " + item_transitions()
        links.append(context_id.add_expr(s, with_ops=True))
        if node_name in room['Drops']:
            s = loc_id(room_name, node_name) + " & " + loc_id(room_name, node_name, when="next") + " & " + item_transitions(room['Drops'][node_name])
            links.append(context_id.add_expr(s, with_ops=True))
    for node_name, door in room['Doors'].items():
        d = loc_id(room_name, node_name) + " & " + loc_id(door['Room'], door['Node'], when="next") + " & " + item_transitions()
        door_bdds.append(context_id.add_expr(d, with_ops=True))
    for node_name, edges in room['Edges'].items():
        for edge in edges:
            other_node_name = edge['Terminal']
            s = loc_id(room_name, node_name) + " & " + loc_id(room_name, other_node_name, when="next") + " & " + required_itemsets(edge['Requirements']) + " & " + item_transitions()
            links.append(context_id.add_expr(s, with_ops=True))
    room_bdd = combine_bdd_group(links)
    #print(room_bdd.count())
    room_bdds.append(room_bdd)

100%|█████████████████████████████████████████████████████████████████████████████████| 255/255 [00:11<00:00, 21.48it/s]


In [19]:
doors_bdd = combine_bdd_group(door_bdds)
rooms_bdd = combine_bdd_group(room_bdds)
all_bdd = doors_bdd | rooms_bdd

In [20]:
all_bdd.dag_size # 11K

11314

In [21]:
context_id.bdd.dump("../output/all_bdd.json", [all_bdd])

In [22]:
all_bdd.count()
# 158,211,025,928,192

158211025928192.0

In [23]:
all_bdd.__dir__()

['__new__',
 '__repr__',
 '__hash__',
 '__str__',
 '__lt__',
 '__le__',
 '__eq__',
 '__ne__',
 '__gt__',
 '__ge__',
 '__invert__',
 '__and__',
 '__rand__',
 '__or__',
 '__ror__',
 '__int__',
 '__len__',
 'implies',
 'equiv',
 'let',
 'exist',
 'forall',
 'pick',
 'count',
 '_index',
 'var',
 'level',
 'ref',
 'low',
 'high',
 'negated',
 'support',
 'dag_size',
 'bdd',
 '_ref',
 '__doc__',
 '__pyx_vtable__',
 '__reduce__',
 '__setstate__',
 '__getattribute__',
 '__setattr__',
 '__delattr__',
 '__init__',
 '__reduce_ex__',
 '__subclasshook__',
 '__init_subclass__',
 '__format__',
 '__sizeof__',
 '__dir__',
 '__class__']

In [24]:
a = context_id.exist(nexts, all_bdd)

In [25]:
a.count() * 2 ** len(set(prevs) - context_id.support(a))

47897475284992.0

In [26]:
context_id.count(all_bdd)

158211025928192

In [27]:
158211025928192 / 47897475284992.0

3.303118274748924

In [28]:
context_id.bdd.count(context_id.exist(nexts, all_bdd), len(prevs))

93549756416.0

In [29]:
from data_types import item_set

In [30]:
def mk_itemset_expr(itemset, when="prev"):
    clauses = []
    for i in item_mapping:
        if i in itemset:
            clause = f"{i}_{when} = 1"
        else:
            clause = f"{i}_{when} = 0"
        clauses.append(clause)
    return " & ".join(clauses)

In [31]:
def get_reachable_id(trans_norule, context, start_bdd):
    n = 0
    covered_p = start_bdd
    covered_last_p = context.false
    while covered_p != covered_last_p:
        covered_last_p = covered_p
        fringe_n = context.exist(prevs, covered_p & trans_norule)
        covered_p |= context.let(next_to_prev, fringe_n)
        print(n, covered_p.dag_size)
        n+=1
    return covered_p

In [32]:
start_bdd = context_id.add_expr("node_id_prev = 2 & " + mk_itemset_expr(item_set.ItemSet()))

In [33]:
%%capture
all_reachable_p = get_reachable_id(all_bdd, context_id, start_bdd)

In [34]:
feasible_trans = all_bdd & all_reachable_p

In [35]:
wincon_p = context_id.add_expr("node_id_prev = 2 & Mother_Brain_prev = 1")

In [36]:
wincon = context_id.add_expr("node_id_goal = 2 & Mother_Brain_goal = 1")

In [37]:
feasible_trans.count()

48840562.0

In [41]:
import json

In [42]:
with open("../output/context_id_order.json", "r") as f:
    context_id.bdd.reorder(json.load(f))
context_id.bdd.configure(reordering=False)

{'reordering': True,
 'garbage_collection': True,
 'max_memory': 18446744073709551615,
 'loose_up_to': 6710886,
 'max_cache_soft': 411648,
 'max_cache_hard': 4294967295,
 'min_hit': 30,
 'max_growth': 1.2,
 'max_swaps': 2000000,
 'max_vars': 1000}

In [43]:
%%time
#TODOs
# Goal BDD x
#   More restrictive goal_bdd using pick w/ transitive closure
#   Forward & backward feasibility restrictions
# ~covered_prev_goal earlier
# Shattergate
all_policy = mk_policy_id3(feasible_trans, context_id, wincon)

0 177 1981 177
1 197 1985 202
2 221 1998 239
3 216 1998 273
4 259 2012 335
5 259 2008 380
6 245 2015 399
7 258 2017 420
8 266 2025 446
9 318 2048 501
10 448 2122 690
11 489 2146 890
12 568 2161 1052
13 722 2211 1262
14 807 2260 1536
15 777 2266 1753
16 858 2277 1847
17 944 2358 2038
18 960 2358 2302
19 970 2425 2535
20 1022 2508 2762
21 1185 2605 2919
22 1342 2647 3217
23 1320 2646 3492
24 1303 2686 3742
25 1302 2791 3864
26 1448 2913 4237
27 1665 3101 4788
28 1657 3109 5179
29 1885 3241 5462
30 1990 3301 6062
31 2303 3431 6650
32 2582 3613 7243
33 2575 3833 8126
34 2902 3995 9012
35 2933 4228 9641
36 3177 4436 10041
37 3664 4530 10962
38 3869 4899 12148
39 4098 4965 12953
40 4305 5132 13476
41 4841 5502 14410
42 4796 5751 14702
43 5505 5958 15854
44 5593 6521 16680
45 6343 6929 17979
46 6402 7191 19029
47 7635 7765 20642
48 8131 8524 23263
49 8576 9271 25253
50 9067 9767 26477
51 9276 10316 27189
52 9787 10502 28993
53 10141 10972 30191
54 10348 11534 31075
55 11328 11850 33207
56 117

In [40]:
# 6:55 - 7:01 - ~6min finished on iteration 604
# Final size 411245 ~ 14MB

In [50]:
all_policy.dag_size

101955

In [51]:
context_id.exist(nexts + prevs, all_policy).count()

9.0

In [56]:
with open("../output/context_id_order.json", "w") as f:
    json.dump(context_id.bdd.var_levels, f)

In [43]:
all_policy = context_id.bdd.load("../output/policy_id.json")[0]

In [44]:
#context_id.bdd.dump("../output/policy_id.dddmp", [all_policy])

In [34]:
goal_satisfied_id = mk_goal_satisfied_id(context_id)

In [35]:
rev_node_ids = {v:k for k,v in node_ids.items()}

In [36]:
# task_bdd is built by first creating a task expr, then existing off the nexts, then picking and cubing it.
def get_path_concrete_id(policy, task_bdd, context):
    # Get a concrete path
    current = context.pick(task_bdd)
    path = [current]
    n = 0
    while True:
        #print(n, current["node_id_prev"])
        #print(n, rev_node_ids[current["node_id_prev"]])
        #print(current["node_id_prev"])
        #print(current)
        n += 1
        #current_bdd = context.bdd.cube(current)
        current_bdd = context.assign_from(current)
        next_temp = context.exist(prevs, current_bdd & policy)
        advice = context.let(next_to_prev, next_temp)
        #print(advice.count())
        current = context.pick(advice)
        # Check whether the goal is satisfied in the state given by the advice.
        if (next_temp & goal_satisfied_id).pick():
            break
        path.append(current)
    return path

# is cube() faster?
def get_path_concrete_id2(policy, task_bdd, context):
    # Get a concrete path
    current = task_bdd.pick()
    path = [current]
    while True:
        #print(current)
        current_bdd = context.bdd.cube(current)
        next_temp = context.exist(prevs, current_bdd & policy)
        advice = context.let(next_to_prev, next_temp)
        #print(advice.count())
        current = advice.pick()
        # Check whether the goal is satisfied in the state given by the advice.
        if (next_temp & goal_satisfied_id).pick():
            break
        path.append(current)
    return path

def get_first_step(policy, task_dict, context):
    # Get a concrete path
    task_bdd = context.assign_from(task_dict)
    return context.pick(task_bdd & policy)

In [37]:
#task_expr = context_id.add_expr("Mother_Brain_prev < Mother_Brain_goal & node_id_prev = 2 & node_id_goal = 2")
task_expr = context_id.add_expr("M_prev < M_goal & node_id_prev = 2 & node_id_goal = 2")
# Concrete task
task = context_id.pick(context_id.exist(nexts, all_policy & task_expr))
task_bdd = context_id.assign_from(task)

In [38]:
#cc = context_id.exist(prevs, all_bdd & context_id.add_expr("node_id_prev = 2"))
#cc = context_id.exist(prevs, all_bdd & task_bdd)
t2 = task.copy()
t2["node_id_prev"] = node_ids["Landing_Site_Ship"]
t2_bdd = context_id.assign_from(t2)
cc = context_id.exist(prevs, all_policy & t2_bdd)

In [39]:
nonidns = [k for k in context_id.vars.keys() if k != "node_id_next"]

In [40]:
for i in context_id.pick_iter(context_id.exist(nonidns, cc)):
    print(rev_node_ids[i["node_id_next"]])

Landing_Site_L2


In [41]:
# 1.94s concrete1
# 0.233s concrete2

In [42]:
%%time
x = get_path_concrete_id2(all_policy, t2_bdd, context_id)

CPU times: user 476 ms, sys: 14 µs, total: 476 ms
Wall time: 476 ms


In [45]:
%%time
x = get_path_concrete_id2(all_policy, task_bdd, context_id)

CPU times: user 486 ms, sys: 32 µs, total: 486 ms
Wall time: 486 ms


In [46]:
len(x)

567

In [1]:
%%time
step = get_first_step(all_policy, task, context_id)

NameError: name 'get_first_step' is not defined

In [262]:
%%time
context_id.assign_from(task)
#task_bdd & all_policy

CPU times: user 1.89 ms, sys: 103 µs, total: 2 ms
Wall time: 2.01 ms


Function (DdNode) with var index: 4, ref count: 2, int repr: 480166083

In [236]:
len(x)

2